<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#До" data-toc-modified-id="До-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>До</a></span></li><li><span><a href="#Преобразование" data-toc-modified-id="Преобразование-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Преобразование</a></span></li><li><span><a href="#После" data-toc-modified-id="После-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>После</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

*Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

План работ:
1. Загрузим и изучим данные (`insurance.csv`).
2. Ответим на вопрос и обоснуем решение.
    Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
        a. Изменится. Приведем примеры матриц.
        b. Не изменится. Укажем, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
3. Предложим алгоритм преобразования данных для решения задачи. Обоснуем, почему качество линейной регрессии не поменяется.
4. Запрограммируем этот алгоритм, применив матричные операции. Проверим, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применим метрику R2.

## Загрузка данных

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
# from sklearn.metrics import mean_squared_error
# import warnings

In [2]:
dt.datetime.today().strftime("%d.%m.%Y %H:%M")

'19.07.2022 12:05'

In [4]:
pth1 = '/datasets/'
pth2 = ''
if os.path.exists(pth1+'insurance.csv'):
    df = pd.read_csv(pth1+'insurance.csv')
elif os.path.exists(pth2+'insurance.csv'):
    df = pd.read_csv(pth2+'insurance.csv')
else:
    print('Something is wrong')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


В таблице 5 столбцов, 5000 строк.
<br>Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
<br>Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [6]:
df.columns

Index(['Пол', 'Возраст', 'Зарплата', 'Члены семьи', 'Страховые выплаты'], dtype='object')

In [7]:
df.columns = ['sex', 'age', 'salary', 'family_members', 'insurance_payments']
df.columns

Index(['sex', 'age', 'salary', 'family_members', 'insurance_payments'], dtype='object')

In [8]:
df.corr()

,sex,age,salary,family_members,insurance_payments
sex,1.000000,0.002074,0.014910,-0.008991,0.010140
age,0.002074,1.000000,-0.019093,-0.006692,0.651030
salary,0.014910,-0.019093,1.000000,-0.030296,-0.014963
family_members,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
insurance_payments,0.010140,0.651030,-0.014963,-0.036290,1.000000


Сильной корреляции между признаками не обнаружено. Но есть средняя корреляция целевого признака с возрастом.

## Умножение матриц

Ответим на вопрос и обоснуем решение.

Признаки умножают на *обратимую* матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


**Ответ: не изменится.**

**Обоснование:**

До преобразования:
$$
MSE=||y-Xw||^2
$$
После преобразования:
$$
MSE_2=||y-X_2w_2||^2,
$$

где 
\begin{equation}
X_2 = X P,
\end{equation}
<br><br>
$$
w_2=(X_2^T X_2)^{-1} X_2^T y = ((XP)^T XP)^{-1} (XP)^T y =
$$

$$
= (P^T X^T X P)^{-1} P^T X^T y = (X^T X P)^{-1} (P^T)^{-1} P^T X^T y = 
$$

$$
= (X^T X P)^{-1} ((P^T)^{-1} P^T) X^T y = (X^T X P)^{-1} E X^T y = 
$$

$$
= (X^T X P)^{-1} X^T y = P^{-1} ((X^T X)^{-1} X^T y) = P^{-1} w
$$

$$
w_2 = P^{-1} w
$$

Можно вывести $w_2$ по-другому. Предсказания должны остаться прежними, чтобы качество регрессии не изменилось:

$$
a = X_2 w_2
$$
$$
X w = X P w_2
$$
$$
w = P w_2
$$
$$
P^{-1} w = P^{-1} P w_2
$$
$$
P^{-1} w = w_2
$$

Таким образом, 
$$
MSE_2 = ||y - (XP) (P^{-1}w)||^2 = 
$$
$$
= ||y - X (P P^{-1}) w||^2 = ||y - X E w||^2 = 
$$
$$
= ||y-Xw||^2 = MSE
$$

<div class="alert alert-success">
<b>✔️ Комментарий ревьювера:</b> Доказательство верное и хорошо оформлено!
    
Матрицы $P^T$ и $P$ извлены из-за скобок правильно. Молодец, что не раскрываешь скобки в выражении $(X^T X)^{-1}$, так как матрицы $X$ и $X^T$ прямоугольные и для них не существует обратных матриц.  
    
Многие делают ошибку в этом месте =)

</div>

## Алгоритм преобразования

Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

**Алгоритм**

Cгенерируем случайную матрицу P подходящей размерности функцией `numpy.random.normal()` *(необратимые матрицы встречаются редко, вероятность получить необратимую матрицу близка к нулю)*, проверим её на обратимость функцией `numpy.linalg.inv()`. Умножим исходную матрицу X на сгенерированную P.

**Обоснование**

При таком преобразовании исходные данные восстановить будет трудно (или невозможно), не зная матрицу P, а качество регрессии не должно измениться, как было показано в п.2 выше.

## Проверка алгоритма

In [9]:
features = df.drop('insurance_payments', axis=1)
target = df['insurance_payments']

model = LinearRegression()

### До

In [10]:
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4249455028666801


### Преобразование

In [11]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
# y = target

In [12]:
rs = np.random.RandomState(np.random.MT19937(np.random.SeedSequence(4754879254)))

P = rs.normal(size=(X.shape[1],X.shape[1]))
P

array([[-0.11470648, -0.0272842 , -0.7586535 , -0.42400872,  0.61477198],
       [-2.51162664,  0.56760216, -0.3757745 , -0.18032687, -0.74563026],
       [-1.34136316,  0.45119326, -0.45708499, -1.13024666, -0.59009744],
       [-0.72963929, -2.77682043,  0.60095691,  0.53874594, -1.95618685],
       [ 0.01709314, -0.64815485,  1.72419958,  0.67256006, -1.27844223]])

In [13]:
try:
    P_inv = np.linalg.inv(P)
    display(P_inv)
except LinAlgError:
    print('P is not square or inversion fails! Try again')

array([[-1.5345849 , -0.51381199,  0.26666397,  0.1313084 , -0.76227732],
       [-1.58347934,  0.03593056,  0.15583681, -0.17497619, -0.58660642],
       [ 2.28831147,  0.04907871, -0.03519395, -0.42332447,  1.73575681],
       [-1.3776418 ,  0.45723499, -0.83208189,  0.15404688, -0.78079314],
       [ 3.14372264,  0.2816461 , -0.56064693, -0.39941887,  1.43521775]])

In [14]:
X2 = X @ P
X2[:3,:]

array([[ -36247.71397652, -137711.90213731,   29789.31195201,
          26675.52654068,  -97052.47087632],
       [ -27788.09338698, -105499.09680948,   22816.3021622 ,
          20420.60277581,  -74362.90831512],
       [ -15361.43935481,  -58300.17166567,   12606.08096176,
          11280.46349571,  -41096.42182786]])

In [15]:
features2 = pd.DataFrame(X2[:,1:], columns=features.columns)
features2.head(3)

,sex,age,salary,family_members
0,-137711.902137,29789.311952,26675.526541,-97052.470876
1,-105499.096809,22816.302162,20420.602776,-74362.908315
2,-58300.171666,12606.080962,11280.463496,-41096.421828


### После

In [16]:
model.fit(features2, target)
predictions = model.predict(features2)
print(r2_score(target, predictions))

0.42494550286668276


### Вывод

`0.4249455028666801`
<br>`0.42494550286668276`

R2 совпали до 15-го знака после запятой. Можно считать, что качество линейной регрессии не изменилось. Эксперимент подтвердил полученный в теории ответ.

In [17]:
dt.datetime.today().strftime("%d.%m.%Y %H:%M")

'19.07.2022 12:06'